# Telemetry Collection Agent (TCA) loads via COA

In [5]:
%disconnect Transcend

Success: 'Transcend' disconnected

In [6]:
%connect TranscendDev

Password: ············


Success: 'TranscendDev' connection established and activated for user 'sh186014'


<br><br><br>
# Build supporting datalab objects 
- Stage tables  (global temp)
- Target Tables (for demo purposes only, will eventually be TCA)
- Stored Procs  (move data from stage to target, for demo purposes only)

<br><br><br>
## DBC.DBCINFO Structures

In [51]:
-- SQL for generating .csv file:
select 
  '{siteid}' as Site_ID
, InfoKey
, InfoData
from dbc.dbcinfo as a

Site_ID,InfoKey,InfoData
{siteid},RELEASE,16.20.53.07
{siteid},VERSION,16.20.53.07
{siteid},LANGUAGE SUPPORT MODE,Standard


In [52]:
database APP_TCA_TMP;

Success: 1 rows affected

In [53]:
drop table stg_tca_dbcinfo

Success: 22 rows affected

In [54]:
/* GLOBAL TEMP TABLE for stage data coming from COA
   note: all users of ADLSTE_COA datalab will need 
         SELECT, INSERT, DELETE, and UPDATE to this object */
CREATE SET GLOBAL TEMPORARY TABLE stg_tca_dbcinfo 
( Site_ID       VARCHAR(128)
, InfoKey       VARCHAR(128)
, InfoData      VARCHAR(128)
) PRIMARY INDEX (Site_ID)
on commit preserve rows

Success: 0 rows affected

In [55]:
drop table coat_tca_dbcinfo

Success: 31 rows affected

In [56]:
-- TARGET TABLE:
CREATE TABLE coat_tca_dbcinfo
( Site_ID        VARCHAR(128)
, InfoKey        VARCHAR(128)
, InfoData       VARCHAR(128)
, di_created_ts  TIMESTAMP(6)   DEFAULT CURRENT_TIMESTAMP(6)
, di_upd_ts      TIMESTAMP(6)   DEFAULT CURRENT_TIMESTAMP(6)
, di_data_src_cd VARCHAR(4)
, di_proc_name   CHAR(30)
, di_batch_nbr   INTEGER
) PRIMARY INDEX(Site_ID);

Success: 0 rows affected

In [57]:
-- ABSTRACTION LAYER VIEW  (** OPTIONAL **) --
replace view ACC_TED_VW.coa_tca_dbcinfo as locking row for access
Select * from coat_tca_dbcinfo;

ERROR: Unable to run SQL: Unable to run SQL query: Database reported error:3524:The user does not have CREATE VIEW access to database ACC_TED_VW.

In [58]:
REPLACE PROCEDURE sp_tca_dbcinfo () 
SQL SECURITY CREATOR
BEGIN     /* this is a simplified example, excluding any audit logging */ 

    DECLARE proc_name CHAR(30);  
    DECLARE batch_nbr INTEGER ;  
    
    /* someone will need to fill these values from ETL */
    SET proc_name = 'Unknown';
    SET batch_nbr = 0        ;   
   
    
    MERGE INTO APP_TCA_TMP.coat_tca_dbcinfo  tgt
         USING APP_TCA_TMP.stg_tca_dbcinfo   stg
            ON (tgt.Site_ID = stg.Site_ID)
    WHEN MATCHED THEN 
        UPDATE SET 
              InfoKey        = stg.InfoKey
            , InfoData       = stg.InfoData
            , di_upd_ts      = Current_Timestamp(6)
            , di_data_src_cd = 'COA'
            , di_proc_name   = :proc_name 
            , di_batch_nbr   = :batch_nbr
    WHEN NOT MATCHED THEN 
        INSERT VALUES 
            ( Site_ID        = stg.Site_ID
            , InfoKey        = stg.InfoKey
            , InfoData       = stg.InfoData
            , di_created_ts  = Current_Timestamp(6)
            , di_upd_ts      = Current_Timestamp(6)
            , di_data_src_cd = 'COA'
            , di_proc_name   = :proc_name 
            , di_batch_nbr   = :batch_nbr ) 
    ;

    delete from APP_TCA_TMP.stg_tca_dbcinfo ;
        
END;

Success: 0 rows affected

In [59]:
-- Let's Test:
insert into stg_tca_dbcinfo
select 
 'Transcend02' as Site_ID
, InfoKey
, InfoData
from dbc.dbcinfo as a;

Success: 3 rows affected

In [60]:
select * from stg_tca_dbcinfo

Site_ID,InfoKey,InfoData
Transcend02,LANGUAGE SUPPORT MODE,Standard
Transcend02,VERSION,16.20.53.07
Transcend02,RELEASE,16.20.53.07


In [62]:
call sp_tca_dbcinfo();

Success: 0 rows affected

In [63]:
Select * from coat_tca_dbcinfo;

Site_ID,InfoKey,InfoData,di_created_ts,di_upd_ts,di_data_src_cd,di_proc_name,di_batch_nbr
Transcend02,LANGUAGE SUPPORT MODE,Standard,2020-11-16 16:01:11.420000,2020-11-16 16:01:11.420000,COA,Unknown,0
Transcend02,VERSION,16.20.53.07,2020-11-16 16:01:11.420000,2020-11-16 16:01:11.420000,COA,Unknown,0
Transcend02,RELEASE,16.20.53.07,2020-11-16 16:01:11.420000,2020-11-16 16:01:11.420000,COA,Unknown,0


<br><br><br>
## ResUsage STAGE -- Build GTT Tables

In [ ]:
drop table adlste_coa_stg."stg_tca_resusagesvpr_pdcr_16.20"

In [ ]:
CREATE GLOBAL TEMPORARY TABLE adlste_coa_stg."stg_tca_resusagesvpr_pdcr_16.20" as
(select cast('' as varchar(128)) as Site_ID, a.* from PDCRInfo.ResUsageSVPR_hst as a) 
with no data
no primary index
ON COMMIT PRESERVE ROWS;

In [ ]:
drop table adlste_coa_stg."stg_tca_resusagesvpr_dbc_16.20"

In [ ]:
CREATE GLOBAL TEMPORARY TABLE adlste_coa_stg."stg_tca_resusagesvpr_dbc_16.20" as
(select cast('' as varchar(128)) as Site_ID, a.* from dbc.ResUsageSVPR as a) 
with no data
no primary index
ON COMMIT PRESERVE ROWS;

In [ ]:
drop table adlste_coa_stg."stg_tca_resusagespma_pdcr_16.20"

In [ ]:
CREATE GLOBAL TEMPORARY TABLE adlste_coa_stg."stg_tca_resusagespma_pdcr_16.20" as
(select cast('' as varchar(128)) as Site_ID, a.* from PDCRInfo.ResUsageSPMA_hst as a) 
with no data
no primary index
ON COMMIT PRESERVE ROWS;

In [ ]:
drop table adlste_coa_stg."stg_tca_resusagespma_dbc_16.20"

In [ ]:
CREATE GLOBAL TEMPORARY TABLE adlste_coa_stg."stg_tca_resusagespma_dbc_16.20" as
(select cast('' as varchar(128)) as Site_ID, a.* from dbc.ResUsageSPMA as a) 
with no data
no primary index
ON COMMIT PRESERVE ROWS;

In [ ]:
drop table adlste_coa_stg."stg_tca_resusagespdsk_pdcr_16.20"

In [ ]:
CREATE GLOBAL TEMPORARY TABLE adlste_coa_stg."stg_tca_resusagespdsk_pdcr_16.20" as
(select cast('' as varchar(128)) as Site_ID, a.* from PDCRInfo.ResUsageSPDSK_hst as a) 
with no data
no primary index
ON COMMIT PRESERVE ROWS;

In [ ]:
drop table adlste_coa_stg."stg_tca_resusagespdsk_dbc_16.20" 

In [ ]:
CREATE GLOBAL TEMPORARY TABLE adlste_coa_stg."stg_tca_resusagespdsk_dbc_16.20" as
(select cast('' as varchar(128)) as Site_ID, a.* from dbc.ResUsageSPDSK as a) 
with no data
no primary index
ON COMMIT PRESERVE ROWS;

<br><br><br>
## ResUsage TARGET -- Normal Tables
this is for demo purposes only... eventually this will reside in another database for TCA upload

### dbcinfo

### resusageSVPR 16.20 PDCR

In [ ]:
drop table adlste_coa."coat_tca_resusagesvpr_pdcr_16.20" 

In [ ]:
CREATE TABLE adlste_coa."coat_tca_resusagesvpr_pdcr_16.20" as
(select cast('' as varchar(128)) as Site_ID, a.*, cast(0 as INT) as Run_ID from PDCRInfo.ResUsageSVPR_hst as a) 
with no data
primary index (Site_ID, TheDate, TheTime);

In [ ]:
replace view adlste_coa."coa_tca_resusagesvpr_pdcr_16.20" as locking row for access
select * from adlste_coa."coat_tca_resusagesvpr_pdcr_16.20" 

### resusageSVPR 16.20 DBC

In [ ]:
drop table adlste_coa."coat_tca_resusagesvpr_dbc_16.20"

In [ ]:
CREATE GLOBAL TEMPORARY TABLE adlste_coa."coat_tca_resusagesvpr_dbc_16.20" as
(select cast('' as varchar(128)) as Site_ID, a.*, cast(0 as INT) as Run_ID from dbc.ResUsageSVPR as a) 
with no data
primary index (Site_ID, TheDate, TheTime);

In [ ]:
replace view  adlste_coa."coa_tca_resusagesvpr_dbc_16.20" as locking row for access
select * from  adlste_coa."coat_tca_resusagesvpr_dbc_16.20"

### resusageSPMA 16.20 PDCR

In [ ]:
drop table adlste_coa."coat_tca_resusagespma_pdcr_16.20"

In [ ]:
CREATE GLOBAL TEMPORARY TABLE adlste_coa."coat_tca_resusagespma_pdcr_16.20" as
(select cast('' as varchar(128)) as Site_ID, a.*, cast(0 as INT) as Run_ID from PDCRInfo.ResUsageSPMA_hst as a) 
with no data
primary index (Site_ID, TheDate, TheTime);

In [ ]:
replace view adlste_coa."coa_tca_resusagespma_pdcr_16.20" as locking row for access
select * from adlste_coa."coat_tca_resusagespma_pdcr_16.20"

### resusageSPMA 16.20 DBC

In [ ]:
drop table adlste_coa_stg."coat_tca_resusagespma_dbc_16.20"

In [ ]:
CREATE GLOBAL TEMPORARY TABLE adlste_coa_stg."coat_tca_resusagespma_dbc_16.20" as
(select cast('' as varchar(128)) as Site_ID, a.*, cast(0 as INT) as Run_ID from dbc.ResUsageSPMA as a) 
with no data
primary index (Site_ID, TheDate, TheTime);

In [ ]:
replace view adlste_coa_stg."coa_tca_resusagespma_dbc_16.20" as locking row for access
select * from adlste_coa_stg."coat_tca_resusagespma_dbc_16.20"

### resusageSPDSK 16.20 PDCR

In [ ]:
drop table adlste_coa."coat_tca_resusagespdsk_pdcr_16.20"

In [ ]:
CREATE GLOBAL TEMPORARY TABLE adlste_coa."coat_tca_resusagespdsk_pdcr_16.20" as
(select cast('' as varchar(128)) as Site_ID, a.*, cast(0 as INT) as Run_ID from PDCRInfo.ResUsageSPDSK_hst as a) 
with no data
primary index (Site_ID, TheDate, TheTime);

In [ ]:
replace view adlste_coa."coa_tca_resusagespdsk_pdcr_16.20" as locking row for access
select * from adlste_coa."coat_tca_resusagespdsk_pdcr_16.20"

### resusageSPDSK 16.20 DBC

In [ ]:
drop table adlste_coa."coat_tca_resusagespdsk_dbc_16.20" 

In [ ]:
CREATE GLOBAL TEMPORARY TABLE adlste_coa."coat_tca_resusagespdsk_dbc_16.20" as
(select cast('' as varchar(128)) as Site_ID, a.*, cast(0 as INT) as Run_ID from dbc.ResUsageSPDSK as a) 
with no data
primary index (Site_ID, TheDate, TheTime);

In [ ]:
replace view adlste_coa."coa_tca_resusagespdsk_dbc_16.20" as locking row for access
select * from adlste_coa."coat_tca_resusagespdsk_dbc_16.20" 

<br><br><br>
## Stored Proc to merge DBC.DBCInfo 
This is for demo purposes... longer term this SP may live elsewhere, or insert into a different target location. <br>
Also note, this SP is templated for COA audit processes, hence the extra logging.

### DBCInfo

In [92]:
REPLACE PROCEDURE adlste_coa.sp_tca_resusage(suffix VARCHAR(16), collection VARCHAR(16), dbsversion VARCHAR(16)) 
SQL SECURITY CREATOR
BEGIN 

    /* start setup audit process */
    DECLARE spversion VARCHAR(128);
    DECLARE runid  INTEGER;
    DECLARE runtype VARCHAR(128);
    DECLARE startdate DATE;
    DECLARE enddate DATE;
    DECLARE siteid VARCHAR(128);
    DECLARE procname VARCHAR(128);
    DECLARE tablename VARCHAR(128);
    DECLARE stagename VARCHAR(128);
    DECLARE callingsp VARCHAR(128);
    DECLARE rowcount bigint;
    DECLARE sqltxt VARCHAR(16000);

    SET procname = 'tca_resusage' || trim(:suffix) ||'_'|| trim(:collection) ||'_'|| trim(:dbsversion);
    SET callingsp = 'adlste_coa.sp_tca_resusage';
    SET tablename = 'adlste_coa.coat_' || procname;    -- "coat_tca_resusagesvpr_dbc_16.20"
    SET stagename = 'adlste_coa_stg.stg_' || procname; -- "stg_tca_resusagespma_pdcr_16.20"
    -------------
    SET spversion = 'v1';
    SET runid = 0;
    SET siteid = 'All';
    SET runtype = 'COA Run, TCA';

    SET startdate = DATE;
    SET enddate = DATE;

    /* collect startdate, enddate, and siteid from Stage */
    SELECT 
     coalesce(Site_ID,'empty table')       
    ,min(TheDate)(DATE) as StartDate
    ,max(TheDate)(DATE) as EndDate
    INTO siteid, startdate, enddate
    FROM adlste_coa_stg.stg_tca_dbcinfo
    Group by 1
    ;
    
    CALL adlste_coa.sp_get_rowcount(:tablename, rowcount);
    CALL adlste_coa.sp_audit_log(runid, :siteid, :tablename, :callingsp, :spversion, :startdate, :enddate, :runtype,
        'START', 'record count: ' || cast(rowcount as varchar(32)) ) ;   /*  run_status, run_msg */
    /* ------------------------
       end setup, begin sp logic
       ------------------------
    */
 
    CALL adlste_coa.sp_audit_log(runid, :siteid, :tablename, :callingsp, :spversion, :startdate, :enddate, :runtype, 
        'STEP 01', 'Delete Duplicates from Target') ;   
        SET sqltxt = ' DELETE FROM ' || :tablename || 
                     ' WHERE (Site_ID, TheDate) IN (Select Site_ID, TheDate From '|| :stagename || ' )' ;

    
    CALL adlste_coa.sp_audit_log(runid, :siteid, :tablename, :callingsp, :spversion,  :startdate, :enddate, :runtype,
        'STEP 02', 'Insert into Target') ;   
        SET sqltxt = ' INSERT into ' || :tablename || ' Select a.*, ' || :runid || ' FROM ' || :stagename ;
    
    
    CALL adlste_coa.sp_audit_log(runid, :siteid, :tablename, :callingsp, :spversion,  :startdate, :enddate, :runtype,
        'STEP 03', 'Delete from GTT') ;   
        SET sqltxt = ' DELETE FROM ' || :stagename ;
                        
            
    CALL adlste_coa.sp_audit_log(runid, :siteid, :tablename, :callingsp, :spversion,  :startdate, :enddate, :runtype,
            'STEP 04', 'Collect Stats') ;   
        set sqltxt = 'collect stats column(Site_ID) on ' || :tablename ;
            
    /* ------------------------
       end sp logic, begin clean-up
       ------------------------
    */   
    CALL adlste_coa.sp_get_rowcount(:tablename, rowcount);
    CALL adlste_coa.sp_audit_log(runid, :siteid, :tablename, :callingsp, :spversion,  :startdate, :enddate, :runtype,
        'END', 'record count: ' || cast(rowcount as varchar(32)) ) ;

END;


ERROR: Unable to run SQL: Unable to run SQL query: Database reported error:5526:Stored Procedure is not created/replaced due to error(s).

<br><br><br>
## Analysis: 
How much data per day?  (assuming 3x compression)

In [ ]:
Select cds.DataBaseName, cds.TableName, 
       sum(CurrentPerm)/1e6 as CurrPerm_MB, max(DateCount) as Date_Cnt,
       (CurrPerm_MB / Date_Cnt * 3.00)(Decimal(18,2)) as MB_Per_Day
from dbc.TableSizeV as cds
join   (select 'PDCRData' as DatabaseName, 'ResUsageSPDSK_Hst' as TableName, 
        count(distinct TheDate) as DateCount from PDCRInfo.ResUsageSPDSK_Hst
        union all
        select 'PDCRData' as DatabaseName, 'ResUsageSVPR_Hst' as TableName, 
        count(distinct TheDate) as DateCount from PDCRInfo.ResUsageSVPR_Hst
        union all
        select 'PDCRData' as DatabaseName, 'ResUsageSPMA_Hst' as TableName, 
        count(distinct TheDate) as DateCount from PDCRInfo.ResUsageSPMA_Hst
        union all
        select 'DBC' as DatabaseName, 'ResUsageSVPR' as TableName, 
        count(distinct TheDate) as DateCount from DBC.ResUsageSVPR
        union all
        select 'DBC' as DatabaseName, 'ResUsageSPMA' as TableName, 
        count(distinct TheDate) as DateCount from DBC.ResUsageSPMA
        union all
        select 'DBC' as DatabaseName, 'ResUsageSPDSK' as TableName, 
        count(distinct TheDate) as DateCount from DBC.ResUsageSPDSK) as cnt
 on trim(cds.DatabaseName) = trim(cnt.DatabaseName)
and trim(cds.TableName) = trim(cnt.TableName)
where cds.DatabaseName in ('PDCRData','DBC')
  and cds.TableName like ANY ('ResUsageSVPR%','ResUsageSPMA%','ResUsageSPDSK%')
Group by 1,2  Order by 1,2

How close are the columns between resuage and the "raw_wla_vw" view?

In [3]:
Select tp as tblPattern, ColumnName, count(*) as TblCount,  min(columnid) as colOrder
,max(case when databasename = 'dbc'        then trim(tablename) else null end) as dbc_table
,max(case when databasename = 'pdcrinfo'   then trim(tablename) else null end) as pdcr_table
,max(case when databasename = 'raw_wla_vw' then trim(tablename) else null end) as wla_table
from dbc.ColumnsV 
cross join (
    select tp, tp||'_hst' as tp1 from (select 'svpr'(varchar(8))  tp) a1 union 
    select tp, tp         as tp1 from (select 'spma'(varchar(8))  tp) a2 union
    select tp, tp||'_hst' as tp1 from (select 'spdsk'(varchar(8)) tp) a3 union
    select tp, tp         as tp1 from (select 'dbcinfo'(varchar(8)) tp) a4 
    ) as t
where (databasename = 'dbc'        and tablename like '%'||tp) 
   or (databasename = 'pdcrinfo'   and tablename like '%'||tp1)
   or (databasename = 'raw_wla_vw' and tablename like '%'||tp||'%hst%1620')
group by 1,2
-- having TblCount <>3
order by 1,4

tblPattern,ColumnName,TblCount,colOrder,dbc_table,pdcr_table,wla_table
dbcinfo,InfoKey,2,1025,DBCInfo,,dbcinfo_hst_1620
dbcinfo,InfoData,2,1026,DBCInfo,,dbcinfo_hst_1620
dbcinfo,thetimestamp,1,1027,,,dbcinfo_hst_1620
dbcinfo,tca_system_id,1,1028,,,dbcinfo_hst_1620
dbcinfo,di_batch_nbr,1,1029,,,dbcinfo_hst_1620
dbcinfo,di_proc_name,1,1030,,,dbcinfo_hst_1620
dbcinfo,di_create_ts,1,1031,,,dbcinfo_hst_1620
dbcinfo,di_update_ts,1,1032,,,dbcinfo_hst_1620
spdsk,TheDate,3,1,ResUsageSpdsk,ResUsageSpdsk_Hst,resusagespdsk_hst_1620
spdsk,NodeID,3,2,ResUsageSpdsk,ResUsageSpdsk_Hst,resusagespdsk_hst_1620


In [2]:
%connect Transcend

Password: ············


Success: 'Transcend' connection established and activated for user 'sh186014'


In [5]:
select *  from tedw.security_dn_4085 where user_logon = 'sh186014'

ERROR: Unable to run SQL: Unable to run SQL query: Database reported error:3523:The user does not have SELECT access to tedw.security_dn_4085.